***WMC via Knowledge compilation***


---


1. Write a method that transforms a formula in sd-DNNF form, and use
this method for computing the weighted model count of propositional
formulas.

2. Check the correctness of your algorithm by comparing the results of
your method with the explicit computation of weighted model
counting via truth table.

3. Write a method that estimates #SAT using the sampleSat algorithm,
and compare the result of the approximated counting with the result
obtained by the exact counting.






In [282]:
#import libraries
from sympy import symbols, simplify, Not, And, Or, Implies, satisfiable, S, simplify, sympify
from sympy import symbols, And, Or, Not, Implies
from sympy.logic.boolalg import to_nnf, Boolean
from sympy.logic.inference import satisfiable
from sympy.logic.boolalg import to_cnf
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from itertools import combinations
from functools import reduce
import time
import math, random
from typing import Dict, Tuple, List
from dataclasses import dataclass

In [283]:
from IPython.display import display, HTML

def render_pipeline(stages):
    # stages: list[(name, done)]
    badges = []
    for name, done in stages:
        icon = "✅" if done else "⏳"
        opacity = "1" if done else ".6"
        badges.append(f"""
        <div style="display:flex; align-items:center; gap:8px; padding:8px 12px;
                    border:1px solid #e5e7eb; border-radius:999px; background:#ffffff; margin:6px;
                    font-family:Inter, Arial; opacity:{opacity};">
            <span style="width:10px; height:10px; border-radius:999px; background:{'#10b981' if done else '#fbbf24'}; display:inline-block;"></span>
            <span style="font-size:14px;">{icon}</span>
            <span style="font-weight:600; font-size:14px;">{name}</span>
        </div>""")
    n_done = sum(1 for _, d in stages if d)
    total = len(stages) or 1
    pct = int(100 * n_done / total)
    bar = f"""
    <div style="height:10px; background:#f0f0f0; border-radius:6px; overflow:hidden; margin-top:6px;">
        <div style="height:10px; width:{pct}%; background:#10b981;"></div>
    </div>"""
    return f"""
    <div>
        <div style="display:flex; flex-wrap:wrap;">{''.join(badges)}</div>
        <div style="font-family:Inter, Arial; margin:6px 2px; font-size:13px;">Progress: {pct}%</div>
        {bar}
    </div>"""

In [284]:
from IPython.display import display, HTML

def render_kpi_cards(results):
    # results: list of dicts with keys: name, exact, approx
    cards = []
    for r in results:
        exact = float(r.get("exact", 0))
        calculated = float(r.get("calculated", 0))
        delta = calculated - exact
        rel = (abs(delta) / exact) if exact != 0 else float("inf")

        def fmt_int(x):
            try:
                x = int(x)
                return f"{x:,}".replace(",", ".")
            except Exception:
                return str(x)

        def fmt_float(x):
            try:
                return f"{x:.4f}"
            except Exception:
                return str(x)

        cards.append(f"""
        <div style="flex:1; min-width:260px; max-width:360px;
                    border:1px solid #e0dede; border-radius:18px; padding:18px; margin:12px;
                    background:linear-gradient(135deg, #fef6f9, #f3f8ff);
                    font-family:Inter, Arial, sans-serif; color:#333; 
                    box-shadow:0 4px 10px rgba(0,0,0,0.05);">
            <div style="font-size:14px; color:#666; font-weight:500;">Test</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('name','N/A')}
            </div>
            <div style="font-size:14px; color:#666; font-weight:500;">Formula</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('formula','N/A')}
            </div>
            <div style="font-size:14px; color:#666; font-weight:500;">sd-DNNF</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('sd-DNNF','N/A')}
            </div>
            <div style="display:flex; gap:14px; align-items:baseline; margin:6px 0;">
                <div style="flex:1; background:#d9f2e6; color:#224; border-radius:12px; padding:10px;">
                    <div style="font-size:12px; opacity:.8;">Exact</div>
                    <div style="font-size:20px; font-weight:700;">{fmt_int(exact)}</div>
                </div>
                <div style="flex:1; background:#ffe6ea; color:#224; border-radius:12px; padding:10px;">
                    <div style="font-size:12px; opacity:.8;">Calculated</div>
                    <div style="font-size:20px; font-weight:700;">{fmt_int(calculated)}</div>
                </div>
            </div>
            <div style="display:flex; gap:14px; margin-top:12px;">
                <div style="flex:1; border:1px dashed #ccc; border-radius:12px; padding:10px; background:#f7fdfb;">
                    <div style="font-size:12px; color:#666;">Δ (calculated - exact)</div>
                    <div style="font-weight:700; color:#333;">{fmt_int(delta)}</div>
                </div>
            </div>
        </div>
        """)

    html = f"""
    <div style="display:flex; flex-wrap:wrap; gap:10px; align-items:stretch;">
        {''.join(cards)}
    </div>
    """
    display(HTML(html))

In [285]:
# ===========================
# Styling pastel
# ===========================
def style_pastel(df: pd.DataFrame):
    df = df.drop(columns=['relative_error','p_hat','n'], errors='ignore')
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    sty = (df.style
           .hide(axis="index")
           .set_table_styles([
               {"selector": "th.col_heading",
                "props": [("background", "#f0f0f0"), ("color", "#374151"),
                          ("font-size", "13px"), ("font-weight", "600"),
                          ("padding", "10px 8px"), ("border", "none")]},
               {"selector": "tbody td",
                "props": [("padding", "10px 12px"), ("border-bottom", "1px solid #e5e7eb"),
                          ("font-size", "13px"), ("color", "#f0f0f0")]},
               {"selector": "table",
                "props": [("border-collapse", "separate"), ("border-spacing", "0"),
                          ("border", "1px solid #e5e7eb"), ("border-radius", "12px"),
                          ("overflow", "hidden"), ("box-shadow", "0 4px 12px rgba(0,0,0,.05)")]},
           ])
           .format(precision=4)
          )

    if num_cols:
        # Gradiente pastello (viola→rosa→azzurro chiaro)
        sty = sty.background_gradient(
            cmap="coolwarm", subset=num_cols,
            vmin=df[num_cols].min().min(),
            vmax=df[num_cols].max().max()
        )

    return sty

Pipeline to convert a formula F into sd-DNNF:
1.	Start with a formula **F**.
2.	Transform it into ***NNF*** (Negation Normal Form): a formula where negations appear only in front of atomic propositions.
3.	Transform it into ***DNNF*** (Decomposable Negation Normal Form): an NNF where each conjunction shares no atomic propositions, i.e., Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ props(Φ₂) = ∅. To achieve this, Shannon’s Expansion is applied.
4.	Transform it into ***d-DNNF*** (Deterministic Decomposable Negation Normal Form): a DNNF where, for every disjunction Φ₁ ∨ Φ₂ ∨ … ∨ Φₙ in the formula, there is at most one i such that I |= Φᵢ.
This ensures that the branches of an OR are mutually exclusive; otherwise, in Weighted Model Counting (WMC), we risk double-counting the same truth assignment.
5.	Transform it into ***sd-DNNF*** (Smooth Deterministic Decomposable Negation Normal Form): apply smoothing (left and right) to guarantee that all disjuncts contain all propositional variables — see the transformation rules (in practice, a tautology is introduced for missing atomic propositions).

Note: Strictly speaking, the very first step would be the transformation of F into CNF (Conjunctive Normal Form): a formula in which implication (→) and equivalence (≡) symbols do not appear. However, in our code this step is handled by the to_nnf function (imported).

In [286]:
# ============================================================
#  F -> NNF -> DNNF -> d-DNNF -> sd-DNNF (con Tau) -> WMC
#  (no normalize_formula: use directly to_nnf)
# ============================================================
# ---------------------------
# Boolean node Tau(v) for visible structural smoothing
# ---------------------------
class Tau(Boolean):
    """
    Represents the tautology (v ∨ ¬v) as an explicit node.
    In WMC, it contributes [w(v) + w(¬v)].
    """
    is_Tau = True

    def __new__(cls, v):
        v = sympify(v)
        obj = Boolean.__new__(cls)
        obj._v = v
        return obj

    @property
    def v(self):
        return self._v

    @property
    def args(self):
        return (self._v,)

    def _hashable_content(self):
        return (self._v,)

    @property
    def free_symbols(self):
        return {self._v}

def _tau(v):
    return Tau(v)

def _vars(e):
    return set(e.free_symbols)

# ---------------------------
# Shannon expansion (NO simplify)
# ---------------------------
def shannon_expansion(f, s):
    """
    f == (s ∧ f|s=True) ∨ (¬s ∧ f|s=False)
    """
    if s not in f.free_symbols:
        return f
    f1 = f.subs({s: S.true})
    f0 = f.subs({s: S.false})
    return Or(And(s, f1), And(Not(s), f0))

# ---------------------------
# Utilities: variable-connected components
# ---------------------------
def _var_components(conj):
    """
    Group the factors of an AND into variable-connected blocks.
    Different blocks do not share variables -> decomposable AND.
    """
    if isinstance(conj, And):
        factors = list(conj.args)
    else:
        factors = [conj]
    comps = []
    while factors:
        seed = factors.pop()
        group = [seed]
        vars_grp = set(seed.free_symbols)
        changed = True
        while changed:
            changed = False
            rest = []
            for f in factors:
                if vars_grp & set(f.free_symbols):
                    group.append(f)
                    vars_grp |= set(f.free_symbols)
                    changed = True
                else:
                    rest.append(f)
            factors = rest
        comps.append(group)
    return comps

# ---------------------------
# 1) NNF -> DNNF
# Function that takes a NNF and returns a DNNF.
# It applies Shannon's Expansion in order to ensure decomposability.
# Every conjunction does not have common atomic propositions, i.e. Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ (Φ₂) = ∅.
# ---------------------------
def NNF2DNNF(nnf):

    #Take Boolean variables from the formula and put them in a list
    #nnf.free_symbols is a set of SymPy symbols that appear in the expression
    atoms = list(nnf.free_symbols)

    #base case: the formula has 0 or 1 variable
    if len(atoms) <= 1:
        return simplify(nnf)

    #I build groups that are “variable-connected”: if I have only one group → not decomposable ⇒ I apply Shannon
    if isinstance(nnf, And): #check if current node is conjunction
        # _var_components takes the factors of the AND (e.g., F1 ∧ F2 ∧ F3) and groups 
        # Them into blocks such that within each block the factors share variables. 
        # If different blocks do not share variables → they are candidates for a decomposable AND.
        # Example: (A∨B)∧(¬A∨C)∧(D∨E) produces groups = [[A∨B, ¬A∨C], [D∨E]].
        groups = _var_components(nnf)
        if len(groups) > 1:
            compiled = [
                NNF2DNNF(And(*g)) if len(g) > 1 else NNF2DNNF(g[0])
                for g in groups ]
            return simplify(And(*compiled))  # decomposable AND 
        else: # No possible decomposition -> force split with Shannon
            pivot = atoms[0]
            return NNF2DNNF(shannon_expansion(nnf, pivot))

    if isinstance(nnf, Or):
        kids = [NNF2DNNF(k) for k in nnf.args]
        return simplify(Or(*kids))          # determinism after
    
    # fallback
    pivot = atoms[0]
    return NNF2DNNF(shannon_expansion(nnf, pivot))

# ---------------------------
# 2) DNNF -> d-DNNF (determinism on OR)
# ---------------------------
def _make_or_deterministic(expr):
    """
    Makes Or(...) deterministic without infinite recursions.
    Strategy:
    - as long as there is a joinable pair of children,
    choose a pivot in the INTERSECTION of their variables (if empty, in the UNION),
    apply Shannon to the entire OR and restart. Iterative.
    """
    assert isinstance(expr, Or)
    # flatten + dedup
    args = list(Or(*expr.args).args)
    uniq = []
    for a in args:
        if a not in uniq:
            uniq.append(a)
    args = uniq
    cur = Or(*args)

    while True:
        n = len(args)
        changed = False

        for i in range(n):
            for j in range(i+1, n):
                if satisfiable(And(args[i], args[j])):  # overlap -> not deterministic
                    Vi, Vj = _vars(args[i]), _vars(args[j])
                    cand = list(Vi & Vj) or list(Vi | Vj)  # first intersection, then union
                    for p in cand:
                        new_or = shannon_expansion(Or(*args), p)
                        if new_or != cur:       # check structural progress
                            cur = new_or
                            args = list(cur.args) if isinstance(cur, Or) else [cur]
                            changed = True
                            break
                    break
            if changed:
                break
        if not changed:
            return cur

# ---------------------------
# 2) DNNF -> d-DNNF (determinism on OR)
# ---------------------------
def DNNF2dDNNF(dnnf):
    if isinstance(dnnf, And):
        return And(*(DNNF2dDNNF(a) for a in dnnf.args))
    if isinstance(dnnf, Or):
        kids = [DNNF2dDNNF(a) for a in dnnf.args]
        det = _make_or_deterministic(Or(*kids))
        if isinstance(det, Or):
            return Or(*(DNNF2dDNNF(a) for a in det.args))
        return det
    #print("dnnf: ", dnnf)
    return dnnf  # literal / True / False

# ---------------------------
# 3) d-DNNF -> sd-DNNF (smoothing with Tau)
# ---------------------------
def ddnnf2sdNNF(ddnnf):
    if isinstance(ddnnf, And):
        return And(*(ddnnf2sdNNF(a) for a in ddnnf.args))
    if isinstance(ddnnf, Or):
        kids = [ddnnf2sdNNF(a) for a in ddnnf.args]
        allv = set().union(*(_vars(k) for k in kids))
        smoothed = []
        for k in kids:
            missing = allv - _vars(k)
            kk = k
            for v in missing:
                kk = And(kk, _tau(v))  # keep Tau(v) visible
            smoothed.append(kk)
        return Or(*smoothed)
    #print("ddnnf: ", ddnnf)
    return ddnnf  # literal / True / False

# ---------------------------
# 4) WMC on sd-DNNF (weights to literals)
# ---------------------------
def model_counting_sdnnf(sdNNF_formula, weights):
    """
    Calculate WMC on sd-DNNF:
      - literal: weights[literal]
      - Tau(v):  weights[v] + weights[Not(v)]
      - AND:     products of children
      - OR:      sum of children
    """
    if sdNNF_formula is S.true:
        return 1.0
    if sdNNF_formula is S.false:
        return 0.0

    # literals (a, ~a, ...)
    if sdNNF_formula in weights:
        #print("sdNNF_formula, ", float(weights[sdNNF_formula]))
        return float(weights[sdNNF_formula])

    # Tau(v) inserted from smoothing
    if isinstance(sdNNF_formula, Tau):
        v = sdNNF_formula.v
        return float(weights[v] + weights[Not(v)])

    # AND: product
    if isinstance(sdNNF_formula, And):
        val = 1.0
        for ch in sdNNF_formula.args:
            val *= model_counting_sdnnf(ch, weights)
        return val

    # OR: sum
    if isinstance(sdNNF_formula, Or):
        return sum(model_counting_sdnnf(ch, weights) for ch in sdNNF_formula.args)

    raise ValueError(f"Unmanaged node: {sdNNF_formula}")

# ---------------------------
# 5) Orchestrator End-to-end: F -> sd-DNNF
# ---------------------------
def compile_to_sdDNNF(F,details):
    """
    F -> NNF -> DNNF -> d-DNNF -> sd-DNNF
    (no final simplify to avoid losing the Tau nodes)
    """
    # removes ⇒, ↔ e force NOT to literals
    
    F_nnf  = to_nnf(F, simplify=True)   
    if details: print("to_NNF: ", F_nnf); time.sleep(2.5)
    stages[0] = ("to NNF", True); handle.update(HTML(render_pipeline(stages)))

    F_dnnf = NNF2DNNF(F_nnf)
    if details: print("F_DNNF: ", F_dnnf); time.sleep(2)
    stages[1] = ("to DNNF", True); handle.update(HTML(render_pipeline(stages)))
    
    F_dd   = DNNF2dDNNF(F_dnnf)
    if details: print("F_dDNNF: ", F_dd); time.sleep(2)
    stages[2] = ("to dDNNF", True); handle.update(HTML(render_pipeline(stages)))
    
    F_sdd  = ddnnf2sdNNF(F_dd)
    if details: print("F_sdNNF: ", F_sdd); time.sleep(2)
    stages[3] = ("to sdNNF", True); handle.update(HTML(render_pipeline(stages)))
    return F_sdd

For other test cases -> modify the formula and weights section

In [ ]:
# ==============================================================
# Approximate #SAT using Monte Carlo + SampleSAT utility
# ==============================================================

# ------------------ Utilities ------------------

def _vars_set(F):
    if "_vars" in globals() and callable(_vars):
        return set(_vars(F))
    return set(F.free_symbols)

def eval_under(F: Boolean, assign: Dict):
    return bool(F.subs(assign))

def random_assignment(V, rng=random):
    return {v: bool(rng.getrandbits(1)) for v in V}

# ---- CNF helpers for SampleSAT ----
def _cnf_clauses(F: Boolean):
    cnf = to_cnf(F, simplify=True)
    if cnf == True:   # tautologia
        return []
    if cnf == False:  # contraddizione
        return [[]]
    if isinstance(cnf, Or):
        return [[arg for arg in cnf.args]]
    if isinstance(cnf, And):
        clauses = []
        for c in cnf.args:
            if isinstance(c, Or):
                clauses.append([arg for arg in c.args])
            else:
                clauses.append([c])
        return clauses
    return [[cnf]]

def _lit_is_pos(lit):
    return getattr(lit, 'is_Symbol', False)

def _lit_var(lit):
    return lit if _lit_is_pos(lit) else lit.args[0]

def _lit_satisfied(lit, A: Dict):
    v = _lit_var(lit)
    val = bool(A.get(v, False))
    return val if _lit_is_pos(lit) else (not val)

def _unsat_clauses(clauses, A):
    return [cl for cl in clauses if not any(_lit_satisfied(l, A) for l in cl)]

def _break_count(lit, clauses, A):
    v = _lit_var(lit)
    A2 = dict(A); A2[v] = not A2[v]
    return sum(1 for cl in clauses if not any(_lit_satisfied(l, A2) for l in cl))

# ------------------ SampleSAT (stochastic local search) ------------------

def samplesat(F: Boolean, max_tries=50, max_flips=1000, noise=0.5, rng=None):
    """
    Returns (found:boolean, assignment:dict).
    IF found=False, assignment is {}.
    """
    rng = rng or random
    V = set(_vars_set(F))
    clauses = _cnf_clauses(F)

    if clauses == []:    # tautologia
        return True, {v: True for v in V}
    if clauses == [[]]:  # contraddizione
        return False, {}

    for _ in range(max_tries):
        A = random_assignment(V, rng)
        for _ in range(max_flips):

            if all(any(_lit_satisfied(l, A) for l in cl) for cl in clauses):
                return True, dict(A)
            
            U = _unsat_clauses(clauses, A)
            cl = rng.choice(U)
            if rng.random() < noise:
                # random flip
                l = rng.choice(cl)
            else:
                # flip “greedy”
                l = min(cl, key=lambda lit: _break_count(lit, clauses, A))
            v = _lit_var(l)
            A[v] = not A[v]
    return False, {}


# --- Exact counting via sd-DNNF ---
def exact_count_sdnnf(F,details=False):
    sd = compile_to_sdDNNF(F,details=False)
    V = _vars_set(F)
    weights = {}
    for v in V:
        weights[v] = 1.0
        weights[Not(v)] = 1.0
    return model_counting_sdnnf(sd, weights)

# ------------------ Approximate model counting (Monte Carlo ) ------------------

def required_samples_eps_delta(eps: float, delta: float) -> int:
    """
    Numero di campioni m tale che |p̂ - p| <= eps con confidenza ≥ 1-delta (Hoeffding).
    """
    if not (0 < eps < 1): raise ValueError("eps deve essere in (0,1).")
    if not (0 < delta < 1): raise ValueError("delta deve essere in (0,1).")
    return math.ceil((1/(2*eps*eps)) * math.log(2/delta))

def approximate_model_count(F: Boolean, eps=0.05, delta=0.05, rng=None) -> Dict:

    rng = rng or random
    V = _vars_set(F)
    n = len(V)
    #m = required_samples_eps_delta(eps, delta)
    m = 2000  # fisso per velocità
    hits = 0
    for _ in range(m):
        A = random_assignment(V, rng)
        hits += int(eval_under(F, A))
    p_hat = hits / m
    estimate = p_hat * (2 ** n)
    return {"n": n, "samples": m, "p_hat": p_hat, "approx_count": estimate, "eps": eps, "delta": delta}

# ------------------ Confronto + tabella ------------------

@dataclass
class CompareResult:
    formula: str
    n: int
    exact: int
    approx: float
    rel_error: float
    p_hat: float
    samples: int

def compare_exact_vs_approx(Fs: List[Tuple[str, Boolean]], eps=0.05, delta=0.05, rng=None,details=False) -> pd.DataFrame:
    rows = []
    for name, F in Fs:
        approx_res = approximate_model_count(F, eps=eps, delta=delta, rng=rng)
        
        try:
            exact = exact_count_sdnnf(F,details=False)
            n = len(_vars_set(F))
        except Exception:
            exact, n = math.nan, approx_res["n"]
        rel_err = (abs(approx_res["approx_count"] - exact) / exact) if (exact == exact and exact != 0) else math.nan
        rows.append(CompareResult(
            formula=name, n=n, exact=exact, approx=approx_res["approx_count"],
            rel_error=rel_err, p_hat=approx_res["p_hat"], samples=approx_res["samples"]
        ).__dict__)
    return pd.DataFrame(rows)


# DEMO

In [288]:
# ===========================
# TEST CASE 1 (Example 7.5 dispensa - result = 136)
# ===========================

# Formula: (a & b) | (c & ¬a)
a, b, c = symbols('a b c')
F1 = Or(And(a, b), And(c, Not(a)))
print("Formula:", F1)

# Weights
weights = {
    a: 2,   Not(a): 1,
    b: 5,   Not(b): 3,
    c: 7,   Not(c): 1
}
stages = [("to NNF", False), ("to DNNF", False), ("to dDNNF", False), ("to sdNNF", False), ("Counting", False)]
handle = display(HTML(render_pipeline(stages)), display_id=True)
# Orchestration till sd-DNNF
F1_sdd = compile_to_sdDNNF(F1,details=True)
print("\nsd-DNNF:", F1_sdd)

# WMC calculation with actual weights
wmc_val = model_counting_sdnnf(F1_sdd, weights)
time.sleep(2)
stages[4] = ("Counting", True); handle.update(HTML(render_pipeline(stages)))
print("\nWMC:", wmc_val)

# Results
results = [
    {"name": "F1", "formula":F1, "sd-DNNF": F1_sdd, "exact": 136, "calculated": wmc_val}
]
render_kpi_cards(results)

Formula: (a & b) | (c & ~a)


to_NNF:  (a & b) | (c & ~a)
F_DNNF:  (a & b) | (c & ~a)
F_dDNNF:  (a & b) | (c & ~a)
F_sdNNF:  (a & b & Tau(c)) | (c & ~a & Tau(b))

sd-DNNF: (a & b & Tau(c)) | (c & ~a & Tau(b))

WMC: 136.0


In [289]:
# ===========================
# TEST CASE 2 — (Exercise 119 dispensa - result = 23)
# ===========================

# Formula: (A ∨ B) → (B ∨ C)
A, B, C = symbols('A B C')
F2 = Implies(Or(A, B), Or(B, C))
print("Formula F2:", F2)

# Literals Weights (from screenshot)
weights = {
    A: 1,  Not(A): 2,
    B: 1,  Not(B): 2,
    C: 1,  Not(C): 2,
}

stages = [("to NNF", False), ("to DNNF", False), ("to dDNNF", False), ("to sdNNF", False), ("Counting", False)]
handle = display(HTML(render_pipeline(stages)), display_id=True)
# Compilazione e WMC
F2_sdd = compile_to_sdDNNF(F2,details=True)
print("\nsd-DNNF:", F2_sdd)

wmc_val = model_counting_sdnnf(F2_sdd, weights)
time.sleep(2)
stages[4] = ("Counting", True); handle.update(HTML(render_pipeline(stages)))
print("\nWMC:", wmc_val)

# Results
results = [
    {"name": "F2", "formula":F2, "sd-DNNF": F2_sdd, "exact": 23, "calculated": wmc_val}
]
render_kpi_cards(results)

Formula F2: Implies(A | B, B | C)


to_NNF:  B | C | (~A & ~B)
F_DNNF:  B | C | ~A
F_dDNNF:  B | (~B & (~A | (A & C)))
F_sdNNF:  (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))

sd-DNNF: (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))

WMC: 23.0


In [293]:

# ==============================================================
# Exact Counting VS Approximate Counting
# ==============================================================

A, B, C = symbols('A B C')

F3 = And(A, B)  # #SAT = 1, n=2
F4 = Or(Not(A), Not(B), Not(C))  # #SAT = 7, n=3

found, assign = samplesat(F1, max_tries=60, max_flips=500, noise=0.5)
print("F1 SampleSAT found: \n", found, "assignment:", assign,"\n")

found, assign = samplesat(F2, max_tries=60, max_flips=500, noise=0.5)
print("F2 SampleSAT found: \n", found, "assignment:", assign,"\n")

found, assign = samplesat(F3, max_tries=60, max_flips=500, noise=0.5)
print("F3 SampleSAT found: \n", found, "assignment:", assign,"\n")

found, assign = samplesat(F4, max_tries=60, max_flips=500, noise=0.5)
print("F4 SampleSAT found: \n", found, "assignment:", assign,"\n")


#Approximated counting and comparison 
formule = [("F1: (a and b) or (c and not a)", F1),
            ("F2: (a and b) or (c and not a)", F2),
            ("F3: a and b", F3),
            ("F4: not a or not b or not c", F4)]

df = compare_exact_vs_approx(formule, eps=0.05, delta=0.05, rng=random.Random(0),details=False)
print(df.to_string(index=False))

F1 SampleSAT found: 
 True assignment: {a: True, c: False, b: True} 

F2 SampleSAT found: 
 True assignment: {A: True, B: True, C: True} 

F3 SampleSAT found: 
 True assignment: {A: True, B: True} 

F4 SampleSAT found: 
 True assignment: {A: False, B: False, C: True} 

                       formula  n  exact  approx  rel_error  p_hat  samples
F1: (a and b) or (c and not a)  3    4.0   4.000   0.000000 0.5000     2000
F2: (a and b) or (c and not a)  3    7.0   7.076   0.010857 0.8845     2000
                   F3: a and b  2    1.0   0.990   0.010000 0.2475     2000
   F4: not a or not b or not c  3    7.0   6.960   0.005714 0.8700     2000
